# 计算图

计算图是用于神经网络BP算法的一种工具，其基本思想是复合函数的链式求导法则，可简化误差反向传播的计算。

## 局部计算节点

计算图将神经网络的推理与学习任务分散各个局部计算节点，通过局部节点的计算结果实现神经网络的推理与学习任务。

- **加法节点**

加法节点的作用是实现以下推理的计算片断，

$$
x + y = z
$$(node-add)

误差的反向传播则将$\frac{\partial L}{\partial z} $乘上以下局部计算结果后，

$$
\begin{split}
\frac{\partial z}{\partial x}&=1\\
\frac{\partial z}{\partial y}&=1\\
\end{split}
$$(node-add-local-comp)

反向传入相应分支，即，$x,y$的各分支分别反向传入$\frac{\partial L}{\partial z}\times 1$。式{eq}`node-add-local-comp`分别对应各个分支的局部梯度计算结果。

- **乘法节点**

与加法节点类似，实现以下局部计算，

$$
x*y=z
$$(node-mult)

误差反向传播则分别将以下结果反向传入对应分支，即$x$分支传入，

$$
\frac{\partial L}{\partial z}\frac{\partial z}{\partial x}=\frac{\partial L}{\partial z}\cdot y
$$(node-mult-back-x)

$y$分支传入，

$$
\frac{\partial L}{\partial z}\frac{\partial z}{\partial y}=\frac{\partial L}{\partial z}\cdot x
$$(node-mult-back-y)

- **分支节点**

分支节点是指相同的值复制后传入各个分支，也称为复制节点。反向传播则是上游传来的梯度之和。当分支扩展到$N$个节点，则可称为**重复节点**。重复节点的反向传播与分支节点类似，是上游所传的梯度之和。



In [10]:
import numpy as np
x = np.random.randn(1, 3)  # 假设的输入数据
y = np.repeat(x, 2, axis=0) # 正向传播

dy=np.random.randn(2, 3) #假设的梯度
dx=np.sum(dy, axis=0, keepdims=True) #梯度传播
print("Input x:\n", x)
print("Output y:\n", y) 
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)

Input x:
 [[-0.80527759 -2.10122899  1.82170405]]
Output y:
 [[-0.80527759 -2.10122899  1.82170405]
 [-0.80527759 -2.10122899  1.82170405]]
Gradient dy:
 [[-0.15132099 -0.00797295  0.24365406]
 [ 0.02775551 -0.69924707  1.85117108]]
Gradient dx:
 [[-0.12356548 -0.70722001  2.09482514]]


- **sum节点**

sum节点与重复节点正好相反，推理时其输入为各个分支的和，反向传播时各分支传入的值是其上游值的复制。

In [11]:
x = np.random.randn(2, 3)  # 假设的输入数据
y = np.sum(x, axis=0, keepdims=True)  # 正向传播
dy = np.random.randn(1, 3)  # 假设的梯度
dx = np.repeat(dy, 2, axis=0)  # 梯度传播
print("Input x:\n", x)  
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)

Input x:
 [[ 0.08492191 -0.91717507 -1.23947848]
 [-0.86875288  0.46808049  0.57779055]]
Output y:
 [[-0.78383097 -0.44909459 -0.66168793]]
Gradient dy:
 [[ 0.95962873 -0.21590781  0.19060428]]
Gradient dx:
 [[ 0.95962873 -0.21590781  0.19060428]
 [ 0.95962873 -0.21590781  0.19060428]]


- **MatMul节点**

矩阵乘积节点(假设向量为行向量)，即

$$
\pmb{y}_{1\times H}=\pmb{x}_{1\times D}\pmb{W}_{D\times H}
$$(node-matmul)

该计算结点的难点在于反向传播，即以下偏导数(**分子布局**)的计算，

$$
\begin{split}
\frac{\partial L}{\partial \pmb{x}}_{1\times D}&=\frac{\partial L}{\partial \pmb{y}}\frac{\partial \pmb{y}}{\partial \pmb{x}}   =\left[\frac{\partial L}{\pmb{y}}\right]_{1\times H}\left[\pmb{W}^\top\right]_{H\times D}\\
\left[\frac{\partial L}{\partial \pmb{W}}\right]_{D\times H}&=\frac{\partial L}{\partial \pmb{y}}\frac{\partial\pmb{y}}{\partial \pmb{W}}   =\left[\pmb{x}^\top\right]_{D\times 1} \left[\frac{\partial L}{\partial\pmb{y}}\right]_{1\times H}\\
\end{split}
$$(node-matmul-back)

式{eq}`node-matmul-back`的第1个等式容易实现，略过。第2个等式的推导如下：由矩阵乘法定义可知，

$$
\frac{\partial y_j}{\partial W_{ik}}=\left\{\begin{array}{ll}x_i,&j==k\\ 0,& j\neq k \end{array} \right.
$$

因此，可以得到以下等式，

$$
\frac{\partial L}{\partial W_{ij}}=\sum_k \frac{\partial L}{\partial y_k}\frac{\partial y_k}{\partial W_{ij}}=\frac{\partial L}{\partial y_j}\frac{\partial y_j}{\partial W_{ij}}=\frac{\partial L}{\partial y_j}\cdot x_i
$$

则有梯度如下，

$$
\frac{\partial L}{\partial \pmb{W}}=\left[ \frac{\partial L}{\partial y_j}x_i  \right]_{ij}=\left[\pmb{x}^\top\right]_{D\times 1} \left[\frac{\partial L}{\partial\pmb{y}}\right]_{1\times H}
$$

注意：当$\pmb{x}$是小批量样本时，{eq}`node-matmul-back`形式仍然保持不变。

In [12]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]  #分子布局
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)  # 矩阵乘法
        self.x = x  #保存输入，反向传播时使用
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)  # 输入梯度
        dW = np.dot(self.x.T, dout)  # 权重梯度
        self.grads[0][...] = dW  # 更新梯度
        return dx
    
# 测试 MatMul 类
W = np.random.randn(3, 4)  # 假设的权重 
x = np.random.randn(2, 3)  # 假设的输入数据
matmul = MatMul(W)  
y = matmul.forward(x)  # 正向传播
dy = np.random.randn(2, 4)  # 假设的梯度    
dx = matmul.backward(dy)  # 反向传播
print("Input x:\n", x)  
print("Weights W:\n", W)
print("Output y:\n", y)     
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)     
print("Gradient dW:\n", matmul.grads[0])  # 权重梯度
    
        

Input x:
 [[ 0.03352755  1.18486835 -1.01735471]
 [ 0.32189715  0.26699387  0.31725705]]
Weights W:
 [[-0.55791106 -0.9511502   0.74444144  1.53739171]
 [ 1.36321165  0.14117636 -0.44070922 -1.09464207]
 [-1.0261097  -0.73398957  0.87890245 -0.29464672]]
Output y:
 [[ 2.64043849  0.88211341 -1.39137866 -0.94570154]
 [-0.14116137 -0.50134268  0.40080492  0.10914054]]
Gradient dy:
 [[ 1.38707728 -2.50204509  0.90305722  0.00865469]
 [ 2.03472731  0.03831168 -1.19331181 -0.85455156]]
Gradient dx:
 [[ 2.2915338   1.13019084  1.20433068]
 [-3.37376827  4.24050429 -2.91298765]]
Gradient dW:
 [[ 0.70147823 -0.07155502 -0.35384637 -0.27478754]
 [ 2.18676369 -2.95436506  0.75139699 -0.21790536]
 [-0.765618    2.55762199 -1.29731611 -0.2799174 ]]


## 局部计算的层

通过计算图的计算结点，可以实现一些神经网络的常用层。这些层一般都是结点的组合结果。

- **sigmoid层**

sigmoid层主要由sigmoid函数组成，即

$$
\begin{split}
y&=\frac{1}{\exp(-x)}\\
\frac{\partial y}{\partial x}&=y(1-y)
\end{split}
$$(sigmoid-layer)



In [13]:
class Sigmoid:
    def __init__(self):
        self.params = []    
        self.grads = []
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))  # Sigmoid 函数
        self.out = out  # 保存输出，反向传播时使用
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1-self.out)  # Sigmoid 的梯度
        return dx

# 测试 Sigmoid 类
x = np.random.randn(2, 3)  # 假设的输入数据
sigmoid = Sigmoid()
y = sigmoid.forward(x)  # 正向传播
dy = np.random.randn(2, 3)  # 假设的梯度
dx = sigmoid.backward(dy)  # 反向传播
print("Input x:\n", x)
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)  # Sigmoid 的梯度


Input x:
 [[-1.65838602  1.02453508 -0.30456766]
 [-1.623109   -1.06858129  0.33342183]]
Output y:
 [[0.15997877 0.73585504 0.42444126]
 [0.16477655 0.25567298 0.58259173]]
Gradient dy:
 [[ 0.06297039  1.46850255  1.18610748]
 [-0.21990096 -0.63435627  0.57927704]]
Gradient dx:
 [[ 0.00846231  0.28543637  0.28975524]
 [-0.03026392 -0.12072073  0.14086778]]


- **仿射层(Affine)**

Affine层主要实现了线性计算的功能，通过矩阵乘法节点和重复节点完成计算功能。

$$
\pmb{z}=\pmb{x}^\top\pmb{W}+\pmb{b}
$$(affine-node)

:::{figure-md}
:name: fig-affine
![仿射节点](../img/affine.svg){width=600px}

仿射节点
:::

参见图{ref}`fig-affine`的计算过程。该层的计算代码实现如下：

In [14]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None   #反向传播时需要
    
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dw = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...]=dw
        self.grads[1][...]=db
        return dx 

#测试Affine节点
W = np.random.randn(3, 4)  # 假设的权重
b = np.random.randn(4)  # 假设的偏置    
x = np.random.randn(2, 3)  # 假设的输入数据
affine = Affine(W, b)
y = affine.forward(x)  # 正向传播
dy = np.random.randn(2, 4)  # 假设的梯度
dx = affine.backward(dy)  # 反向传播
print("Input x:\n", x)
print("Weights W:\n", W)
print("Bias b:\n", b)
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)  # 输入梯度
print("Gradient dW:\n", affine.grads[0])  # 权重梯度
print("Gradient db:\n", affine.grads[1])  # 偏置梯度

    

Input x:
 [[ 0.88704974 -0.30459424  0.49883379]
 [-0.93483483 -1.51573514  1.48494939]]
Weights W:
 [[-1.07665424  0.04425788  0.35114002  0.13897377]
 [-1.80324798  1.1501323   0.05891955  0.71797076]
 [ 0.52208198 -0.04464385 -0.96533174 -0.11528565]]
Bias b:
 [ 1.12997141 -0.82246972  1.78742753 -1.235317  ]
Output y:
 [[ 0.98461663 -1.15580432  1.59941955 -1.38823849]
 [ 5.64497694 -2.67343332 -0.0636056  -2.62468139]]
Gradient dy:
 [[-0.24752332 -1.01983307  0.9752888   0.84968113]
 [-0.31528368 -0.48409676  0.31856986  0.61283347]]
Gradient dx:
 [[ 0.6819077  -0.05908725 -1.12313146]
 [ 0.51505682  0.47052585 -0.52116849]]
Gradient dW:
 [[ 0.07517267 -0.45209215  0.56731947  0.18081135]
 [ 0.55328073  1.04439775 -0.77993489 -1.1877012 ]
 [-0.5916533  -1.22758638  0.95956713  1.33387634]]
Gradient db:
 [-0.562807   -1.50392983  1.29385866  1.4625146 ]


- **Softmax损失层**

该层主要由softmax函数以及交叉熵损失函数复合而成。softmax函数是指以下函数，

$$
softmax(\pmb{x})=\frac{\exp(\pmb{x})}{\sum_i \exp(x_i)}
$$(softmax-fun-def)

交叉熵损失则是指以下损失函数(单个样本one-hot形式)，

$$
loss(y,t)=\sum_i t_i\log y_i
$$(corss-entropy-def)

In [15]:
def softmax(x):
    if x.ndim == 2:  # 如果是二维数组
        x -= np.max(x, axis=1, keepdims=True)  # 减去每行的最大值
        y = np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)  # softmax计算
    else:  # 如果是一维数组
        x -= np.max(x)  # 减去最大值
        y = np.exp(x) / np.sum(np.exp(x))  # softmax计算
    return y

def cross_entropy_loss(y, t):
    if y.ndim == 1:  # 如果是向量
        y = y.reshape(1, -1)  # 转换为二维数组
        t = t.reshape(1, -1)  # 转换为二维数组
    if t.size == y.size:
        t = t.argmax(axis=1)  # 如果t是one-hot编码，转换为类别索引
    batch_size = y.shape[0]  # 获取批大小
    loss = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size  # 计算交叉熵损失
    return loss

class SoftmaxWithLoss:
    def __init__(self):
        self.params = []
        self.grads = []
        self.y = None  # 保存softmax输出
        self.t = None  # 保存标签
    
    def forward(self, x, t):
        self.t = t  # 保存标签
        self.y = softmax(x)  # 计算softmax输出

        # 在监督标签为one-hot向量的情况下，转换为正确解标签的索引
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
        loss = cross_entropy_loss(self.y, self.t)
        return self.y
    
    def backward(self, dout=1):
        batch_size = self.y.shape[0]  # 获取批大小
        dx = self.y.copy()  # 复制softmax输出
        dx[np.arange(batch_size), self.t] -= 1  # 减去正确类别的梯度
        dx *= dout
        dx /= batch_size  # 平均化梯度
        return dx
    
# 测试 SoftmaxWithLoss 类
x = np.random.randn(2, 3)  # 假设的输入数据
t = np.array([0, 1])  # 假设的标签  
softmax_loss = SoftmaxWithLoss()
y = softmax_loss.forward(x, t)  # 正向传播
dy = softmax_loss.backward()  # 反向传播
print("Input x:\n", x)
print("Labels t:\n", t)
print("Softmax Output y:\n", y)
print("Gradient dy:\n", dy)  # softmax的梯度
# 测试 SoftmaxWithLoss 类的交叉熵损失
loss = cross_entropy_loss(y, t)  # 计算交叉熵损失
print("Cross Entropy Loss:\n", loss)  # 输出交叉熵损失
# 测试 SoftmaxWithLoss 类的梯度
print("Gradient dx:\n", dy)  # 输出梯度


Input x:
 [[ 0.         -1.59407251 -0.47112775]
 [-2.35106312  0.         -3.12348144]]
Labels t:
 [0 1]
Softmax Output y:
 [[0.54722717 0.11114009 0.34163273]
 [0.0836217  0.87775387 0.03862442]]
Gradient dy:
 [[-0.22638641  0.05557005  0.17081637]
 [ 0.04181085 -0.06112306  0.01931221]]
Cross Entropy Loss:
 0.36664000348434594
Gradient dx:
 [[-0.22638641  0.05557005  0.17081637]
 [ 0.04181085 -0.06112306  0.01931221]]
